### [탁송기사/화물 AI 매칭 서비스]
#### ↓ 최적의 운행시각 경로 추천 시스템 ↓
--------------------------------------

In [13]:
import pandas as pd
import requests
import folium
import webbrowser
import os
from datetime import datetime, timedelta
import random
import re
import sys

# ======================================================
# 🔑 카카오 REST API KEY
# ======================================================
KAKAO_REST_API_KEY = "17cac59d2ab47302628e667cc96e6877"

HEADERS = {
    "Authorization": f"KakaoAK {KAKAO_REST_API_KEY}"
}

# ======================================================
# 📍 주소 정규화
# ======================================================
def normalize_address(addr):
    if pd.isna(addr):
        return None
    addr = re.sub(r"\d+-?\d*$", "", addr)  # 번지 제거
    addr = addr.replace("인근", "").strip()
    return addr

# ======================================================
# 📍 주소 → 좌표 (자동 + 키워드 + 보정)
# ======================================================
fallback_cache = {}

def kakao_address_search(query):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    res = requests.get(url, headers=HEADERS, params={"query": query}).json()
    if res["documents"]:
        d = res["documents"][0]
        return float(d["y"]), float(d["x"])
    return None

def kakao_keyword_search(query):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    res = requests.get(url, headers=HEADERS, params={"query": query}).json()
    if res["documents"]:
        d = res["documents"][0]
        return float(d["y"]), float(d["x"])
    return None

def address_variants(address):
    parts = address.split()
    variants = []

    # 전체 주소
    variants.append(address)

    # 점점 축소
    if len(parts) >= 3:
        variants.append(" ".join(parts[:3]))   # 시 군 면
    if len(parts) >= 2:
        variants.append(" ".join(parts[:2]))   # 시 군
    if len(parts) >= 1:
        variants.append(parts[0])               # 시

    return list(dict.fromkeys(variants))  # 중복 제거

def address_to_coord_auto(address):
    if address in fallback_cache:
        return fallback_cache[address]

    for query in address_variants(address):
        coord = kakao_address_search(query)
        if coord:
            fallback_cache[address] = coord
            return coord

        coord = kakao_keyword_search(query)
        if coord:
            fallback_cache[address] = coord
            return coord

    return None

# ======================================================
# 🛣️ 실제 도로 경로 (OSRM)
# ======================================================
def get_real_route(origin, destination):
    lon1, lat1 = origin[1], origin[0]
    lon2, lat2 = destination[1], destination[0]

    url = (
        f"http://router.project-osrm.org/route/v1/driving/"
        f"{lon1},{lat1};{lon2},{lat2}"
        "?overview=full&geometries=geojson"
    )

    data = requests.get(url).json()
    if data.get("code") != "Ok":
        return None, None, None

    route = data["routes"][0]
    coords = [(lat, lon) for lon, lat in route["geometry"]["coordinates"]]
    distance_km = round(route["distance"] / 1000, 1)
    duration_min = int(route["duration"] / 60)

    return coords, distance_km, duration_min

# ======================================================
# 🌦️ 랜덤 기상
# ======================================================
def random_weather():
    return random.choice(["☀️ 맑음", "⛅ 흐림", "☔ 비", "❄️ 눈", "🌫️ 안개"])

# ======================================================
# 🧾 팝업 카드 UI
# ======================================================
def popup_card(title, rows):
    html = f"""
    <div style="
        width:340px;
        font-size:14px;
        line-height:1.6;
        word-break: keep-all;
        white-space: normal;
    ">
        <div style="
            font-weight:bold;
            font-size:16px;
            margin-bottom:8px;
        ">{title}</div>

        <table style="width:100%; border-collapse:collapse;">
    """
    for k, v in rows.items():
        html += f"""
        <tr>
            <td style="padding:4px 6px; font-weight:bold; width:35%; vertical-align:top;">
                {k}
            </td>
            <td style="padding:4px 6px;">
                {v}
            </td>
        </tr>
        """
    html += "</table></div>"
    return html

# ======================================================
# 🗺️ 지도 표시
# ======================================================
def show_map(row):
    start = address_to_coord_auto(row["출발지주소"])
    end = address_to_coord_auto(row["도착지주소"])

    if not start or not end:
        print("❌ 좌표 변환 실패")
        return

    route, dist_km, time_min = get_real_route(start, end)
    if route is None:
        print("❌ 도로 경로 생성 실패")
        return

    weather = random_weather()

    m = folium.Map(location=route[0], zoom_start=7, tiles="OpenStreetMap")

    # 출발지 마커
    folium.Marker(
        start,
        popup=folium.Popup(
            popup_card("🚩 출발지 정보", {
                "주소": row["출발지주소"],
                "연락처": row["출발지연락처"],
                "출발시간": row["출발시간"]
            }),
            max_width=400
        ),
        icon=folium.Icon(color="green", icon="play")
    ).add_to(m)

    # 도착지 마커
    folium.Marker(
        end,
        popup=folium.Popup(
            popup_card("🏁 도착지 정보", {
                "주소": row["도착지주소"],
                "연락처": row["도착지연락처"],
                "특이사항": row["특이사항"]
            }),
            max_width=400
        ),
        icon=folium.Icon(color="red", icon="flag")
    ).add_to(m)

    # 경로
    poly = folium.PolyLine(route, color="blue", weight=6, opacity=0.9).add_to(m)
    m.fit_bounds(poly.get_bounds())

    # 경로 정보 마커
    folium.Marker(
        route[len(route)//2],
        popup=folium.Popup(
            popup_card("🚚 탁송 운행 정보", {
                "차종": row["차종"],
                "출발시간": row["출발시간"],
                "기상": weather,
                "거리": f"{dist_km} km",
                "예상시간": f"{time_min} 분",
                "금액": f"{int(row['탁송금액']):,} 원",
                "특이사항": row["특이사항"]
            }),
            max_width=420
        ),
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

    path = os.path.abspath("dispatch_map.html")
    m.save(path)
    webbrowser.open(f"file://{path}")

# ======================================================
# 📂 엑셀 로드 & 추천 10개
# ======================================================
df = pd.read_excel("delivery_orders_2026.xlsx")

print("\n🚩 기사 최초 출발지 주소 입력:")
start_input = input("👉 ")

# ❌ 최초 출발지 주소가 데이터에 전혀 없으면 종료
if not df["출발지주소"].astype(str).str.contains(start_input).any():
    print("❌ 입력한 출발지 주소와 일치하는 데이터가 없습니다. 프로그램을 종료합니다.")
    sys.exit(0)


# 출발지 키워드(앞 2단어 기준)
tokens = start_input.split()
keywords = tokens[:2]

def is_near(addr):
    return all(k in addr for k in keywords)

recommended = df[df["출발지주소"].apply(is_near)].head(10)

print("\n📋 추천 탁송 목록 (10개)")
print(recommended[["인덱스", "차종", "출발지주소", "도착지주소", "탁송금액"]].to_string(index=False))

sel = input("\n👉 선택할 인덱스: ")

row = recommended[recommended["인덱스"] == sel]
if row.empty:
    print("❌ 인덱스 없음")
    sys.exit(0)

row = row.iloc[0]

if pd.isna(row["출발시간"]):
    row["출발시간"] = (
        datetime.now() + timedelta(minutes=30)
    ).strftime("%Y-%m-%d %H:%M")

# ======================================================
# 🔁 랜덤 도착지 선택 (자기 자신 제외)
# ======================================================
def get_random_destination(df, exclude_addr):
    candidates = df["출발지주소"].dropna().unique().tolist()
    candidates = [a for a in candidates if a != exclude_addr]

    if not candidates:
        return None

    return random.choice(candidates)

# 지도 표시
show_map(row)
print("✅ 지도 표시 완료.")

# ======================================================
# 🔄 엑셀 데이터 업데이트
# ======================================================
selected_idx = row.name  # DataFrame index

old_start = row["출발지주소"]
old_end = row["도착지주소"]

# 1️⃣ 도착지 → 다음 출발지
new_start = old_end

# 2️⃣ 랜덤 도착지 선택
new_end = get_random_destination(df, new_start)

if new_end is None:
    print("❌ 랜덤 도착지 선택 실패")
    sys.exit(0)

# 3️⃣ DataFrame 업데이트
df.at[selected_idx, "출발지주소"] = new_start
df.at[selected_idx, "도착지주소"] = new_end

# (선택) 출발시간 초기화
df.at[selected_idx, "출발시간"] = (
    datetime.now() + timedelta(minutes=30)
).strftime("%Y-%m-%d %H:%M")

# 4️⃣ 엑셀 저장
df.to_excel("delivery_orders_2026.xlsx", index=False)

print("🔁 엑셀 업데이트 완료")
print(f"   출발지: {new_start}")
print(f"   도착지: {new_end}")


🚩 기사 최초 출발지 주소 입력:


👉  인천시광역시 연수구 해돋이로120번길 23



📋 추천 탁송 목록 (10개)
   인덱스      차종                           출발지주소                도착지주소   탁송금액
  26_6 제네시스g90         인천시광역시 연수구 해돋이로120번길 23 경상북도 문경시 문경읍 새재로 475 250000
 26_75      클릭         인천시광역시 연수구 해돋이로120번길 23   강원 홍천군 서면 한치골길 782 240000
26_167     그랜저 인천시광역시 연수구 인천시타워대로 99 305호 송도중고  전남 여수시 돌산읍 돌산로 2980 240000



👉 선택할 인덱스:  ㅇ


❌ 인덱스 없음


SystemExit: 0

C:\Users\user\miniconda3\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
